In [ ]:
%matplotlib inline

import cv2
import datacube
import geopandas as gpd
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import rasterio
import seaborn as sns
import sys
import time

sys.path.insert(1, '../Tools/')

from datetime import datetime, timedelta
from dea_tools.plotting import display_map, rgb
from math import ceil
from odc.ui import with_ui_cbk, DcViewer
from rasterio.features import geometry_mask
from rasterio.transform import from_origin
from shapely.geometry import Polygon, mapping
from skimage.transform import rescale
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from tifffile import imsave
from tqdm import tqdm

In [ ]:
# Set some configurations for displaying tables nicely
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", None)

In [ ]:
dc = datacube.Datacube(app="Full_S2_Image_WB")

In [ ]:
# Definition of the product
set_product = "ga_s2am_ard_3"
measurements = dc.list_measurements()

In [ ]:
# Definition of the parameters
    
# Set the measurements/bands to load
set_measurements = [
    "red",
    "blue",
    "green",
    "red_edge_1",
    "red_edge_2",
    "nir_1",
    "swir_2",
    "swir_3",
]

# Set of the resolution for resampling
set_resolution = (10,10)

In [ ]:

# Set the date range to load data over
set_time = ('2023-02-07', '2023-02-09')

# Fowlers Gap coordinates
# lon_min = 540758
# lon_max = 589824
# lat_min = 6546069
# lat_max = 6587164

# Witchelina coordinates
lon_min = 186171
lon_max = 229989
lat_min = 6625457
lat_max = 6693958

study_area_lon = (lon_min, lon_max)
study_area_lat = (lat_min, lat_max)

# Load of the specific dataset
ds = dc.load(
    product=set_product,
    x=study_area_lon, 
    y=study_area_lat,
    time=set_time,
    measurements=set_measurements,
    crs="EPSG:32754",
    output_crs="EPSG:32754",
    resolution=set_resolution,
    resampling={"red_edge_1":'bilinear', "red_edge_2":'bilinear', "swir_2":'bilinear', "swir_3":'bilinear'}
)


if len(ds.time) > 1:
    ds = ds.isel(time=3)
    rgb(ds, bands=['red', 'green', 'blue']) 
else:
    rgb(ds, bands=['red', 'green', 'blue'], col="time") 



In [ ]:
def ND(B1,B2):
    ND = (B1-B2) / (B1+B2)
    return ND

def Div(B1):
    Div = B1 / 10000
    return Div


# Activate this part for the Fowlers Gap image
# y_max = ds.dims['y'] - 1  # Récupérer la dimension y et soustraire 1 pour obtenir la valeur maximale
# ds_inversed = ds.isel(y=slice(y_max, None, -1))  # Inversion des coordonnées y

# Activate this part for the Witchelina image
ds_inversed = ds

# Définition de chaque bande
B2 = Div(ds_inversed.blue) 
B3 = Div(ds_inversed.green)
B4 = Div(ds_inversed.red) 
B5 = Div(ds_inversed.red_edge_1) 
B6 = Div(ds_inversed.red_edge_2) 
B8 = Div(ds_inversed.nir_1) 
B11 = Div(ds_inversed.swir_2) 
B12 = Div(ds_inversed.swir_3) 

# Calcul of the MNDWI
ds_MNDWI = ND(B3,B11)

# Calcul of the MuWI-C
# MuWIR = -4*ND(B2,B3) + 2*ND(B3,B8) + 2*ND(B3,B12) - ND(B3,B11)
ds_MuWI = -16.4*ND(B2,B3) - 6.9*ND(B2,B4) - 8.2*ND(B2,B8) - 8.8*ND(B2,B11) + 9.6*ND(B2,B12) + 10.8*ND(B3,B8) + 6.1*ND(B3,B11) + 13.6*ND(B3,B12) - 0.28*ND(B4,B8) - 3.9*ND(B4,B11) - 2.1*ND(B4,B12) - 5.3*ND(B8,B11) - 5.3*ND(B8,B12) - 5.3*ND(B11,B12) -0.33

# Calcul of the TWI
ds_TWI = ((2.84 * (B5-B6)) / (B3+B12)) + ((1.25 * (B3-B2) - (B8-B2)) / (B8 + 1.25*B3 - 0.25*B2))

# Calcul of the WI
ds_WI = 1.7204 + 171*B3 + 3*B4 - 70*B8 - 45*B11 - 71*B12

# Calcul of the albedo
albedo = (B2 + B3 + B4 + B5 + B6 + B8 + B11 + B12) / 8

# index_to_save = ds_WI

# index_to_save = np.squeeze(index_to_save) 
# resolution = 10  
# transform = from_origin(lon_min, lat_max, resolution, resolution)

# output_path_WI = "Water_Polygons_Project/Images/Witchelina_Calcul_WI_-14_2023.tif"
# with rasterio.open(output_path_WI, 'w', driver='GTiff', width=index_to_save.shape[1], height=index_to_save.shape[0], count=1, dtype=index_to_save.dtype, crs='EPSG:32754', transform=transform) as dst_WI:
#     dst_WI.write(index_to_save, 1)

# plt.imshow(ds_WI, cmap="RdBu")
# plt.show()

In [ ]:
threshold_MNDWI = -0.3
classified_MNDWI = np.where(ds_MNDWI < threshold_MNDWI, 0, 1)

threshold_MuWI = 3.0
classified_MuWI = np.where(ds_MuWI < threshold_MuWI, 0, 1)

threshold_TWI = -0.25
classified_TWI = np.where(ds_TWI < threshold_TWI, 0, 1)

threshold_WI = -14.0
classified_WI = np.where(ds_WI < threshold_WI, 0, 1)

classified_MNDWI = np.squeeze(classified_MNDWI)  
classified_MuWI = np.squeeze(classified_MuWI)  
classified_TWI = np.squeeze(classified_TWI)  
classified_WI = np.squeeze(classified_WI)  

plt.imshow(classified_TWI, cmap='gray')
plt.colorbar() 
plt.show()

resolution = 10  
transform = from_origin(lon_min, lat_max, resolution, resolution)

# output_path_MNDWI = "Water_Polygons_Project/Images/Fowlers_Gap_Classified_MNDWI_{threshold_MNDWI}.tif"
# with rasterio.open(output_path_MNDWI, 'w', driver='GTiff', width=classified_MNDWI.shape[1], height=classified_MNDWI.shape[0], count=1, dtype=classified_MNDWI.dtype, crs='EPSG:32754', transform=transform) as dst_MNDWI:
#     dst_MNDWI.write(classified_MNDWI, 1)

# output_path_MuWI = "Water_Polygons_Project/Images/Fowlers_Gap_Classified_MuWI_{threshold_MuWI}.tif"
# with rasterio.open(output_path_MuWI, 'w', driver='GTiff', width=classified_MuWI.shape[1], height=classified_MuWI.shape[0], count=1, dtype=classified_MuWI.dtype, crs='EPSG:32754', transform=transform) as dst_MuWI:
#     dst_MuWI.write(classified_MuWI, 1)

# output_path_TWI = "Water_Polygons_Project/Images/Fowlers_Gap_Classified_TWI_{threshold_TWI}.tif"
# with rasterio.open(output_path_TWI, 'w', driver='GTiff', width=classified_TWI.shape[1], height=classified_TWI.shape[0], count=1, dtype=classified_TWI.dtype, crs='EPSG:32754', transform=transform) as dst_TWI:
#     dst_TWI.write(classified_TWI, 1)

output_path_WI = f"Water_Polygons_Project/Images/Witchelina_Classified_WI_{threshold_WI}.tif"
with rasterio.open(output_path_WI, 'w', driver='GTiff', width=classified_WI.shape[1], height=classified_WI.shape[0], count=1, dtype=classified_WI.dtype, crs='EPSG:32754', transform=transform) as dst_WI:
    dst_WI.write(classified_WI, 1)
